In [3]:
import numpy as np
import pandas as pd
import difflib # get the closest match to the actual movie if the user misspelled the movie name
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
movies_data = pd.read_csv('movies.csv')

In [5]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
movies_data.shape

(4803, 24)

In [7]:
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [8]:
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [9]:
# combining all 5 selected features
combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [10]:
# converting text data into feature vector
vectorizer = TfidfVectorizer()
feature_vector = vectorizer.fit_transform(combined_features)
print(feature_vector)

  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.24713765026964
  (4801, 17266)	0.28860981849329476
  (4801, 13835)	0.27870029291200094
  (4801, 13175)	0.28860981849329476
  (4801, 171

In [11]:
# getting the similarity score using cosine similiarity
similarity = cosine_similarity(feature_vector)
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [12]:
similarity.shape

(4803, 4803)

In [13]:
# getting the movie name from user
movie_name = input('Enter your favorite movie name: ')

Enter your favorite movie name: Star Wars


In [14]:
# creating a list with all movie names given in the dataset
movies = movies_data['title'].tolist()
print(movies)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [15]:
# finding the closest match for the given movie name
find_close_match = difflib.get_close_matches(movie_name, movies)
print(find_close_match)

['Star Wars', 'Star Trek', 'State Fair']


In [16]:
close_match = find_close_match[0]
print(close_match)

Star Wars


In [17]:
# finding the index of the movie with title
index = movies_data[movies_data.title == close_match]['index'].values[0]
print(index)

2912


In [18]:
# getting a list of similar movies
similarity_score = list(enumerate(similarity[index])) # enumerate creates a loop in a list
print(similarity_score)

[(0, 0.03076834408963385), (1, 0.01684407631590081), (2, 0.012588894325688157), (3, 0.005928760033092409), (4, 0.04898677562810463), (5, 0.012746312626083042), (6, 0.007079562626506406), (7, 0.04785189345134744), (8, 0.007224385230671699), (9, 0.012133969127804015), (10, 0.0346073036482771), (11, 0.01093050987754538), (12, 0.012563339421908766), (13, 0.011342950205312523), (14, 0.030567853240903258), (15, 0.006678031119662606), (16, 0.04537376686628985), (17, 0.012018292649658705), (18, 0.08239912076896568), (19, 0.02395795551598849), (20, 0.01225325217182898), (21, 0.024290816343118096), (22, 0.02015592073973481), (23, 0.0060482212036870155), (24, 0.026063263727214972), (25, 0.03530223669119933), (26, 0.03205316628922021), (27, 0.04198647184787163), (28, 0.030771909426408756), (29, 0.012873642862390015), (30, 0.02224715393120497), (31, 0.03023752014989672), (32, 0.015509787201008172), (33, 0.03086368823326212), (34, 0.0), (35, 0.0323991508378306), (36, 0.044982269855707954), (37, 0.01

In [19]:
# sorting the movies based on their similarity score
sorted_similar_movies = sorted(similarity_score, key=lambda x:x[1], reverse=True) # sorted in descending order using a key to sort using the second index (the decimal)
print(sorted_similar_movies)

[(2912, 1.0000000000000002), (1990, 0.2827270485514309), (1490, 0.25723501595447257), (3251, 0.1953019623437404), (539, 0.16284197998767536), (233, 0.15628304180676847), (4401, 0.13295048152265548), (229, 0.12979987977501956), (1715, 0.1125398932451733), (2550, 0.11051313620968847), (670, 0.10573786160144674), (1725, 0.10389200700588526), (3203, 0.10026936270365754), (230, 0.09965476666730125), (430, 0.09927819715045705), (466, 0.09752857307719623), (4251, 0.09577117795078624), (2455, 0.09535848056654629), (1165, 0.09426419125679927), (1515, 0.09425472404112305), (1269, 0.09079165316370223), (2127, 0.09005633673691213), (540, 0.08936487375945953), (351, 0.08819122413615699), (2085, 0.08720501131563961), (260, 0.08719339610579918), (2118, 0.08666647336810646), (4350, 0.08430458890661423), (1103, 0.0835471821868589), (2403, 0.08347430547789392), (18, 0.08239912076896568), (2292, 0.08214595787604803), (53, 0.08209515256571193), (1306, 0.08159036669895915), (3706, 0.08089957345654139), (49

In [20]:
# print the name of similiar movies based on the index
print('Movies suggested for you: \n')
i=1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.', title_from_index)
    i+=1

Movies suggested for you: 

1 . Star Wars
2 . The Empire Strikes Back
3 . Return of the Jedi
4 . On Her Majesty's Secret Service
5 . Titan A.E.
6 . Star Wars: Episode I - The Phantom Menace
7 . The Helix... Loaded
8 . Star Wars: Episode III - Revenge of the Sith
9 . Timecop
10 . Lawrence of Arabia
11 . Random Hearts
12 . Blade Runner
13 . Ultramarines: A Warhammer 40,000 Movie
14 . Star Wars: Episode II - Attack of the Clones
15 . Lilo & Stitch
16 . The Time Machine
17 . Fetching Cody
18 . When Harry Met Sally...
19 . Back to the Future Part III
20 . Laws of Attraction
21 . Raise the Titanic
22 . Bill & Ted's Bogus Journey
23 . Hollywood Homicide
24 . The Departed
25 . Raiders of the Lost Ark
26 . Ender's Game
27 . Premonition
28 . An Inconvenient Truth
29 . The Fugitive
